# Supporting material for "Glaberish: Generalizing the Continuously-Valued Lenia Framework to Arbitrary Life-Like Cellular Automata" -- static figures

* Read the paper: [https://arxiv.org/abs/2205.10463](https://arxiv.org/abs/2205.10463)
* Check out the repository: [https://github.com/rivesunder/yuca](https://github.com/rivesunder/yuca)

Some cells take a significant amount of wall time to execute, which may be a problem for free cloud notebooks like mybinder.org. For shorter execution times, experiment with changing the number of simulation steps or CA grid dimensions. In this notebook, these cells can take a bit longer to run: 
   
* All figures that depend on simulation for computing spatial entropy of CA with random uniform initial grid states (7,8,9,10)
* Figure 11 (gliders in s613 and Hydrogeminium CA) can also take a bit longer to run. 

In [ ]:
# this section needs to be run for setup on colab
# uncomment and run the commands below if on colab
# then restart the runtime and run tests in the next cell


#! git clone https://github.com/riveSunder/yuca.git my_yuca
#%cd my_yuca
#! pip install -e .

In [ ]:
#! python -m testing.test_all

In [ ]:
# make  cells wider
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
import os
import copy
import numpy as np
import scipy
import torch

import matplotlib
import matplotlib.animation 
import matplotlib.pyplot as plt

my_cmap = plt.get_cmap("magma")
plt.rcParams['figure.dpi'] = 300

matplotlib.rcParams["pdf.fonttype"] = 42
matplotlib.rcParams["ps.fonttype"] = 42
#matplotlib.rcParams["animation.embed_limit"] = 256
matplotlib.rcParams["font.size"] = 10
#matplotlib.rcParams["text.usetex"] = True

import IPython

import skimage
import skimage.io as sio
import skimage.transform

import notebooks.my_helpers as my_helpers
from importlib import reload
reload(my_helpers)

import yuca
from yuca.utils import seed_all
from yuca.patterns import get_puffer
from yuca.multiverse import CA
from yuca.lenia import Lenia
from yuca.zoo.librarian import Librarian


from yuca.analyze import get_spatial_entropy, \
        get_multiconditional_grid_entropy, \
        get_conditional_grid_entropy,\
        get_grid_entropy

def get_center_of_mass(grid, spacing=20):
    
    if len(grid.shape) > 2:
        my_grid = np.array(grid.cpu().squeeze())
    else:
        my_grid = np.array(1.0 * grid.cpu())
        
    edge_l = my_grid[:,:spacing].sum()
    edge_r = my_grid[:,-spacing:].sum()
    edge_t = my_grid[:spacing,:].sum()
    edge_b = my_grid[-spacing:,:].sum()
    
    max_edge = np.max([edge_l, edge_r, edge_t, edge_b])
    
    xx_grid, yy_grid = np.meshgrid(np.arange(my_grid.shape[-1]), np.arange(my_grid.shape[-2]))
    
    if max_edge:
        
        if edge_r == max_edge:
            com_x = xx_grid[:, spacing:] * my_grid[:, spacing:] / my_grid[:, spacing:].sum()
            com_y = yy_grid[:, spacing:] * my_grid[:, spacing:] / my_grid[:, spacing:].sum()
            
        elif edge_l == max_edge:
            com_x = xx_grid[:, :-spacing] * my_grid[:, :-spacing] /  my_grid[:, :-spacing].sum()
            com_y = yy_grid[:, :-spacing] * my_grid[:, :-spacing] /  my_grid[:, :-spacing].sum()
            
        elif edge_t == max_edge:
            com_x = xx_grid[:-spacing, :] * my_grid[:-spacing, :] / my_grid[:-spacing, :].sum()
            com_y = yy_grid[:-spacing, :] * my_grid[:-spacing, :] /my_grid[:-spacing, :].sum()
            
        else: # edge_b == max_edge:
            com_x = xx_grid[spacing:, :] * my_grid[spacing:, :] / my_grid[spacing:, :].sum()
            com_y = yy_grid[spacing:, :] * my_grid[spacing:, :] / my_grid[spacing:, :].sum()
            
    else:
        # pattern is not at an edge crossing
        
        com_x = (xx_grid * my_grid) / my_grid.sum()
        com_y = (yy_grid * my_grid) / my_grid.sum()
    
    return com_x.sum(), com_y.sum()


In [ ]:
# execute this cell for gpu support
my_device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print(my_device)

# Figure 1: Hydrogeminium natans (Lenia) and s613 (Glaberish) CA and spatial entropy after 47 steps. 

In [ ]:
# generate the grids and simulate from random initial grid state

ca_g = CA()
ca_s = CA()
ca_g.restore_config("geminium.npy")
ca_s.restore_config("s613.npy")

ca_g.no_grad()
ca_s.no_grad()

ca_g.to_device(my_device)
ca_s.to_device(my_device)

num_steps = 47
dim = 256
channels = 5

for my_seed in [13]:#, 1337, 42]:
    seed_all(my_seed)
    
    g0_teaser = torch.zeros(channels,1, dim//2, dim)
    s1_teaser = torch.zeros(channels,1, dim//2, dim)
    
    rand_init = torch.rand(channels,1,dim//2, dim //4)
    
    g0_teaser[:,:,:,dim//8:3*dim//8] = 1. * rand_init
    
    s1_teaser[:,:,:,dim//8:3*dim//8] = 1. * rand_init
    
    g0_teaser = g0_teaser.to(my_device)
    s1_teaser = s1_teaser.to(my_device)
    
    for step in range(num_steps):
        
        #print((g0_teaser - ca_g(g0_teaser)).mean())
        #print((s1_teaser - ca_s(s1_teaser)).mean())
        g0_teaser = ca_g(g0_teaser)
        #s1_teaser = ca_s(s1_teaser)
        #s1_teaser = ca_s(s1_teaser)
        #s1_teaser = ca_s(s1_teaser)
        s1_teaser = ca_s(s1_teaser)
        
    for ch in range(g0_teaser.shape[0]):
        
        #hg = yuca.analyze.get_spatial_entropy(g0_teaser[ch].cpu().numpy().squeeze(), window_size=15)
        #hs = yuca.analyze.get_spatial_entropy(s1_teaser[ch].cpu().numpy().squeeze(), window_size=15)
        
        fig, ax = plt.subplots(1,2, figsize=(2,3))
        
        disp_g = 1.0 - my_cmap(g0_teaser[ch].cpu().numpy().squeeze())[:,:,:3]
        disp_s = 1.0 - my_cmap(s1_teaser[ch].cpu().numpy().squeeze())[:,:,:3]
        
        ax[0].imshow(disp_g)
        
        plt.title(f"Hydrogeminium seed {my_seed} channel {ch}", fontsize=5)
        ax[1].imshow(disp_s)
        
        #ax[1,0].imshow(hg)
        #ax[1,1].imshow(hs)
        plt.title(f"s613 seed {my_seed} channel {ch}", fontsize=5)

        ax[0].set_xticklabels("")
        ax[1].set_xticklabels("")
        ax[0].set_yticklabels("")
        ax[1].set_yticklabels("")

        plt.show()
        
s1_teaser = s1_teaser[ch]
g0_teaser = g0_teaser[ch]

In [ ]:
disp_g0t = 1.0 - my_cmap(g0_teaser.cpu().numpy().squeeze())[:,:,:3]
disp_s1t = 1.0 - my_cmap(s1_teaser.cpu().numpy().squeeze())[:,:,:3]

h_g0t = yuca.analyze.get_spatial_entropy(g0_teaser.cpu().numpy().squeeze(), window_size=23)
h_s1t = yuca.analyze.get_spatial_entropy(s1_teaser.cpu().numpy().squeeze(), window_size=23)

fig, ax = plt.subplots(2, 4, figsize=(3.9,1.7), gridspec_kw={'width_ratios': [0.25, 2, 2, 0.25]})

ax[0,1].imshow(disp_g0t)
ax[1,1].imshow(disp_s1t)

ax[0,2].imshow(h_g0t, vmin=0,  vmax=8, cmap="magma")
ax[1,2].imshow(h_s1t, vmin=0, vmax=8, cmap="magma")


my_height = disp_g0t.shape[0] 
my_colorbar = (np.arange(my_height, 0, -1) / my_height).reshape(my_height, 1) * np.ones((my_height, my_height//10))

for xxyy in range(8):
    xx = xxyy // 4
    yy = xxyy % 4
    ax[xx,yy].set_xticklabels("")
    ax[xx,yy].set_yticklabels("")
    
ax[0,1].set_title("a", fontsize=8)
ax[0,2].set_title("b", fontsize=8)

ax[1,1].set_xlabel("c", fontsize=8)
ax[1,2].set_xlabel("d", fontsize=8)
        
ax[0,0].set_yticks([my_height, my_height//2, 0])
ax[0,0].set_yticklabels([0, 0.5, 1.0], fontsize=6)

ax[1,0].set_yticks([my_height, my_height//2, 0])
ax[1,0].set_yticklabels([0,0.5, 1.0], fontsize=6)

my_colorbar1 = my_cmap(my_colorbar)#[:,:,:3]
my_colorbar2 = 1.0 - my_cmap(my_colorbar)[:,:,:3]

ax[0,0].imshow(my_colorbar2)
ax[1,0].imshow(my_colorbar2)

ax[0,3].imshow(my_colorbar1)
ax[1,3].imshow(my_colorbar1)

ax[1,3].set_yticks([my_height, my_height//2, 0])
ax[1,3].set_yticklabels([0,4,8], fontsize=6)
ax[0,3].set_yticks([my_height, my_height//2, 0])
ax[0,3].set_yticklabels([0,4,8], fontsize=6)

# pointer arrows call out frog glider
ax[1,1].annotate("←", (130,80) ,color="black", fontsize=12)
ax[1,2].annotate("←",  (130,80), color="white", fontsize=12)

    
#plt.savefig("../assets/glaberish/teaser_figure.png", bbox_inches="tight")
#plt.tight_layout()
plt.show()



In [ ]:
# zoom in to the glider/mobile pattern in s613 

plt.figure(figsize=(3,6)); 
plt.subplot(121);
plt.xticks(fontsize=4)
plt.yticks(fontsize=4)
plt.imshow(disp_s1t[56:86, 96:126])

plt.tick_params(left = False, bottom=False)
plt.subplot(122);

plt.xticks(fontsize=4)
plt.yticks(fontsize=4)
plt.imshow(h_s1t[56:86, 96:126], cmap="magma");# plt.colorbar()

plt.tick_params(left = False, bottom=False)

plt.show()

print(h_s1t[56:86, 96:126].max(), h_s1t[56:86, 96:126].min(), \
      h_s1t[56:86, 96:126].mean(), h_s1t[56:86, 96:126].std())

plt.figure(figsize=(3,6)); 
plt.subplot(121);
plt.xticks(fontsize=4)
plt.yticks(fontsize=4)
plt.imshow(disp_g0t[56:86, 96:126])

plt.tick_params(left = False, bottom=False)
plt.subplot(122);

plt.xticks(fontsize=4)
plt.yticks(fontsize=4)
plt.imshow(h_g0t[56:86, 96:126], cmap="magma");# plt.colorbar()

plt.tick_params(left = False, bottom=False)

plt.show()


print(h_g0t[56:86, 96:126].max(), h_g0t[56:86, 96:126].min(), \
      h_g0t[56:86, 96:126].mean(), h_g0t[56:86, 96:126].std())

In [ ]:
h_s1t[:,:dim//2].std(), h_g0t[:,:dim//2].std()

# Figure 2: Lenia neighborhood kernel and growth function for Orbium CA

In [ ]:
reload(my_helpers)

# Orbium Lenia, 
ca = CA()
ca.restore_config("orbium.npy")
ca.no_grad()

kernel_lenia = ca.neighborhood_kernels[0].squeeze()
ca_config = ca.make_config()
mu = ca_config["genesis_config"]["parameters"][0]
sigma = ca_config["genesis_config"]["parameters"][1]

lenia_growth = lambda x: my_helpers.gaussian(x, mu=mu, sigma=sigma) * 2 - 1

fig = my_helpers.plot_kernel_growth(kernel_lenia, lenia_growth, vmin=0, vmax=kernel_lenia.max(), \
        titles=["$K_{Orbium}$", "${\it Orbium}$ growth function"], invert=True)

# uncomment to save figure
plt.tight_layout()
#plt.savefig("../assets/glaberish/lenia_orbium.png", bbox_inches = "tight")
plt.show()

# Figure 3: Conway’s Game of Life implemented in the Lenia framework

In [ ]:
reload(my_helpers)
# Orbium Lenia, k
ca = CA()
ca.restore_config("conway_life.npy")

kernel_life = ca.neighborhood_kernels.squeeze()


life_birth = lambda x: ca.genesis_fns[0](torch.tensor(x)).numpy()
life_survival = lambda x: ca.persistence_fns[0](torch.tensor(x)).numpy()

fig = my_helpers.plot_kernel_genper(kernel_life, life_birth, life_survival, vmin=0, vmax=kernel_life.max(), \
        titles=["$K_{Moore}$ ", "Life B/S functions"], invert=True, show_combined=True,\
        my_xrange=[[-0.1, 1.25],[-.1, 1.25]])

# uncomment to save figure
plt.tight_layout()
#plt.savefig("../assets/glaberish/conway_life.png", bbox_inches = "tight")
plt.show()

# Figure 4: Continuous implementations of the Life-like Morley/Move CA (B368/S245)

In [ ]:
reload(my_helpers)
# Orbium Lenia, k
ca = CA()
ca.restore_config("morley.npy")

kernel_life = ca.neighborhood_kernels.squeeze()


morley_birth = lambda x: ca.genesis_fns[0](torch.tensor(x)).numpy()
morley_survival = lambda x: ca.persistence_fns[0](torch.tensor(x)).numpy()

growth_fn = lambda x: morley_birth(x)/2 + morley_survival(x)/2

fig = my_helpers.plot_kernel_growth(kernel_life, growth_fn, vmin=0, vmax=kernel_life.max(), \
        titles=["$K_{Moore}$ ", "Morley growth function"], invert=True, my_xrange=[-0.75,1.1])


fig[1][0].set_ylabel("a", rotation=0, fontsize=14) #annotate("a", (-.75,0), fontsize=14)
# uncomment to save figure
plt.tight_layout()
#plt.savefig("../assets/glaberish/morley_in_lenia.png", bbox_inches = "tight")
plt.show()

In [ ]:
reload(my_helpers)
# Orbium Lenia, k

ca = CA()
ca.restore_config("morley.npy")

kernel_life = ca.neighborhood_kernels.squeeze()


morley_birth = lambda x: ca.genesis_fns[0](torch.tensor(x)).numpy()
morley_survival = lambda x: ca.persistence_fns[0](torch.tensor(x)).numpy()

growth_fn = lambda x: morley_birth(x) #+ morley_survival(x)/2

fig = my_helpers.plot_kernel_growth(kernel_life, growth_fn, vmin=0, vmax=kernel_life.max(), \
        titles=["$K_{Moore}$ \n inactive cell", "Morley genesis (B)"], invert=True,  my_xrange=[-0.75,1.1])


fig[1][0].set_ylabel("b", rotation=0, fontsize=14) #annotate("a", (-.75,0), fontsize=14)
plt.tight_layout()
#plt.savefig("../assets/glaberish/morley_genesis.png", bbox_inches = "tight")

growth_fn = lambda x: morley_survival(x)

fig = my_helpers.plot_kernel_growth(kernel_life, growth_fn, vmin=0, vmax=kernel_life.max(), \
        titles=["$K_{Moore}$, \n active cell ", "Morley persistence (S)"], invert=True, cmap_offset=100, my_xrange=[-0.75,1.1])

fig[1][1].plot([2],[2],'o', markersize=5)
# uncomment to save figure

fig[1][0].set_ylabel("c", rotation=0, fontsize=14) #annotate("a", (-.75,0), fontsize=14)
#fig[1][0].set_xlabel("c", fontsize=14)
plt.tight_layout()
#plt.savefig("../assets/glaberish/morley_persistence.png", bbox_inches = "tight")
plt.show()

# Figure 5: A common puffer in Life-like CA Morley/Move (B368/S245)

In [ ]:
# generate figures for Morley Timelapse
torch.set_default_dtype(torch.float32)
puffer = get_puffer()

dim = 26
grid = np.zeros((dim,dim))

grid[8:8+puffer.shape[-2], 5:5+puffer.shape[-1]] = puffer

timelapse = np.zeros((dim*3, dim*3))


ca = CA()

ca.load_config(yuca.configs.get_life_like_config(birth=[3,6,8], survival=[2,4,5]))

for jj in range(3):
    for ii in range(3):
        
        timelapse[(jj % 3) * dim:(1+jj%3)*dim, ii*dim:(1+ii)*dim] = grid * (4 + ii * jj) / 8
        
        timelapse[jj*dim,:] = 0.1
        timelapse[:,ii*dim] = 0.1
        
        for step in range(19):
            print
            grid = ca(torch.tensor(grid, dtype=torch.get_default_dtype()).reshape(1,1,dim,dim)).numpy()[0,0]
        


timelapse[-1,:] = 0.1
timelapse[:,-1] = 0.1

puffer_img = np.zeros((8,10))
puffer = get_puffer()

puffer_img = puffer



puffer_img = (1.0 - my_cmap(puffer_img))[:,:,:3]
timelapse_img = (1.0 - my_cmap(timelapse))[:,:,:3]

fig, ax = plt.subplots(1,2, figsize=(3.2,2.2))
ax[0].imshow(puffer_img)
ax[0].set_title("Morley Puffer in Lenia\n (all time steps)", fontsize=8)

ax[0].set_yticklabels("")
ax[0].set_xticklabels("")

ax[1].imshow(timelapse_img)
ax[1].set_title("Morley Puffer \n Timelapse in Glaberish", fontsize=8)
ax[1].set_yticklabels("")
ax[1].set_xticklabels("")
#plt.savefig("assets/morley_timelapse.png")

for ii in range(3):
    for jj in range(3):
    
        xx = 1.5+ii * 26
        yy = 5.5+jj * 26
        note = f"{(ii + 3*jj)*19}"
        
        ax[1].annotate(note, xy= (xx, yy), fontsize=6)
        
ax[0].set_xlabel("a", fontsize=14)

ax[1].set_xlabel("b", fontsize=14)
#plt.savefig("../assets/glaberish/morley_timelapse.png")
plt.show()

# Figure 6: Rule/neighborhood visualization in Lenia and Glaberish

In [ ]:
reload(my_helpers)
# geminium 

ca = CA()
ca.restore_config("geminium.npy")

ca.no_grad()
kernel_geminium = ca.neighborhood_kernels[0].squeeze()
ca_config = ca.make_config()
mu = ca_config["genesis_config"]["parameters"][0]
sigma = ca_config["genesis_config"]["parameters"][1]

geminium_growth = lambda x: my_helpers.gaussian(x, mu=mu, sigma=sigma) * 2 - 1

fig = my_helpers.plot_kernel_growth(kernel_geminium, geminium_growth, vmin=0, vmax=kernel_geminium.max(), \
        titles=["$K_{Hydrogeminium}$", "Growth function"], \
                                    invert=True)

# uncomment to save figure
plt.tight_layout()

fig[1][0].set_ylabel("a", rotation=0, fontsize=14) #annotate("a", (-.75,0), fontsize=14)
#plt.savefig("../assets/glaberish/lenia_geminium.png", bbox_inches = "tight")


ca = CA()
ca.restore_config("s613.npy")
ca.no_grad()
kernel_geminium = ca.neighborhood_kernels[0].squeeze()
ca_config = ca.make_config()
mu = ca_config["genesis_config"]["parameters"][0]
sigma = ca_config["genesis_config"]["parameters"][1]

genesis_fn = lambda x: ca.genesis_fns[0](torch.tensor(x).clone()).numpy()
persistence_fn= lambda x: ca.persistence_fns[0](torch.tensor(x).clone()).numpy()

fig = my_helpers.plot_kernel_genper(kernel_geminium, genesis_fn, persistence_fn, vmin=0, vmax=kernel_geminium.max(), \
        titles=["$K_{Hydrogeminium}$", "Genesis and persistence"], \
                                    invert=True)

# uncomment to save figure
plt.tight_layout()

fig[1][0].set_ylabel("b", rotation=0, fontsize=14) #annotate("a", (-.75,0), fontsize=14)

#plt.savefig("../assets/glaberish/glaberish_s613.png", bbox_inches = "tight")
plt.show()

# Figures 7, 8, 9, and 10, based on simulation of CA from random uniform starting grid states.

In [ ]:
# Hydrogeminium comparison against s613
seed_all(42)

mean_gem = []
mean_s613 = []
h_gem = []
h_s613 = []
h_cond_gem = []
h_cond_s613 = []
h_cond_step = []


my_cmap2 = plt.get_cmap("gray")


# try shrinking the dim, using fewer steps, 
# or running fewer iterations for faster execution
dim = 256
steps = 512
calculate_every = steps // 10
padding = 1
num_iter = 8

mean_gems = []
mean_s613s = []
h_gems = []
h_s613s = []
h_cond_gems = []
h_cond_s613s = []
h_cond_gems_2 = []
h_cond_s613s_2 = []
h_baselines = []

spatial_h0s = []
spatial_h1s = []

reload(yuca.analyze)

for iteration in range(num_iter):

    mean_gem = []
    mean_s613 = []
    h_gem = []
    h_s613 = []
    h_cond_gem = []
    h_cond_s613 = []
    h_cond_step = []

    spatial_h0 = []
    spatial_h1 = []
    
    bounded_grid = np.zeros((dim, dim))

    bounds = dim // 4
    bounds_2 = dim // 3

    bounded_grid[bounds:-bounds, bounds-padding:bounds+padding] = 0.5
    bounded_grid[bounds:-bounds, -bounds-padding:-bounds+padding] = 0.5
    bounded_grid[bounds-padding:bounds+padding, bounds:-bounds] = 0.5
    bounded_grid[-bounds-padding:-bounds+padding, bounds:-bounds] = 0.5

    ca_0 = CA()
    ca_1 = CA()

    ca_0.restore_config("geminium.npy")
    ca_1.restore_config("s613.npy")

    ca_0.no_grad()
    ca_1.no_grad()

    ca_0.to_device(my_device)
    ca_1.to_device(my_device)

    grid_0 = torch.zeros(1,1,dim,dim)
    grid_0[:,:,bounds_2:-bounds_2, bounds_2:-bounds_2] = torch.rand(1,1, dim-2*bounds_2, dim-2*bounds_2)

    input_grid = 1.0 * grid_0

    grid_0 = grid_0.to(my_device)
    grid_1 = 1.0 * grid_0 #torch.rand(1,1,dim,dim)

    grid_0_start = 1.0 * grid_0
    grid_1_start = 1.0 * grid_1

    h_baselines.append(get_multiconditional_grid_entropy(input_grid.cpu().numpy().squeeze()))

    for step in range(steps):

        if step % calculate_every == 0 or step == steps-1:
            
            print("calc entropy-based metrics")
            
            # image entropy
            h_gem.append(get_grid_entropy(grid_0.cpu().numpy().squeeze()))
            h_s613.append(get_grid_entropy(grid_1.cpu().numpy().squeeze()))
            
            # multi-conditional entropy (avg. entropy conditioned on orthogonal neighbors)
            h_cond_gem.append(get_multiconditional_grid_entropy(grid_0.cpu().numpy().squeeze()))
            h_cond_s613.append(get_multiconditional_grid_entropy(grid_1.cpu().numpy().squeeze()))
            h_cond_step.append(step)
            
            # spatial entropy (moving window image entropy)
            spatial_h0.append(yuca.analyze.get_spatial_entropy(grid_0.cpu().numpy().squeeze(),\
                    window_size=ca_0.neighborhood_kernels.shape[-1]))
            spatial_h1.append(yuca.analyze.get_spatial_entropy(grid_1.cpu().numpy().squeeze(),\
                    window_size=ca_1.neighborhood_kernels.shape[-1]))

        grid_0 = ca_0(grid_0)
        grid_1 = ca_1(grid_1)

        mean_gem.append(grid_0.mean().cpu().numpy())
        mean_s613.append(grid_1.mean().cpu().numpy())
        
    grid_0_mid = 1.0 * grid_0
    grid_1_mid = 1.0 * grid_1
    

    for step in range(steps):
        grid_0 = ca_0(grid_0)
        grid_1 = ca_1(grid_1)
        
    grid_0_end = 1.0 * grid_0
    grid_1_end = 1.0 * grid_1

    mean_gbu = copy.deepcopy(mean_gem)
    mean_sbu = copy.deepcopy(mean_s613)

    mean_gem = [elem / np.mean(mean_gem) for elem in mean_gem]
    mean_s613 = [elem / np.mean(mean_s613) for elem in mean_s613]

    h_cond_gem_2 = [elem - h_baselines[0] for elem in h_cond_gem]
    h_cond_s613_2 = [elem - h_baselines[0] for elem in h_cond_s613]

    mean_gems.append(mean_gem)
    mean_s613s.append(mean_s613)
    h_gems.append(h_gem)
    h_s613s.append(h_s613)
    
    h_cond_gems.append(h_cond_gem)
    h_cond_s613s.append(h_cond_s613)
    h_cond_gems_2.append(h_cond_gem_2)
    h_cond_s613s_2.append(h_cond_s613_2)
    

    spatial_h0s.append(spatial_h0)
    spatial_h1s.append(spatial_h1)


In [ ]:

fig, ax = plt.subplots(3,3, figsize=(3.25,4.5), gridspec_kw={'width_ratios': [0.25, 2, 2]})
    
disp_grid_0_start = 1.0 - my_cmap(grid_0_start.cpu().squeeze())[:,:,:3] - my_cmap2(bounded_grid)[:,:,:3]
disp_grid_1_start = 1.0 - my_cmap(grid_1_start.cpu().squeeze())[:,:,:3] - my_cmap2(bounded_grid)[:,:,:3]

disp_grid_0_mid = 1.0 - my_cmap(grid_0_mid.cpu().squeeze())[:,:,:3] - my_cmap2(bounded_grid)[:,:,:3]
disp_grid_1_mid = 1.0 - my_cmap(grid_1_mid.cpu().squeeze())[:,:,:3] - my_cmap2(bounded_grid)[:,:,:3]

disp_grid_0_end = 1.0 - my_cmap(grid_0_end.cpu().squeeze())[:,:,:3] - my_cmap2(bounded_grid)[:,:,:3]
disp_grid_1_end = 1.0 - my_cmap(grid_1_end.cpu().squeeze())[:,:,:3] - my_cmap2(bounded_grid)[:,:,:3]

my_colorbar = np.arange(1024,0,-1).reshape(1024,1)/1024. * np.ones((1024,96))

ax[0,0].imshow(1.0 - my_cmap(my_colorbar)[:,:,:3])
ax[0,1].imshow(disp_grid_0_start, interpolation="nearest")
ax[0,2].imshow(disp_grid_1_start, interpolation="nearest")
#ax[0,3].imshow(my_cmap(my_colorbar)[:,:,:3])

ax[1,0].imshow(1.0 - my_cmap(my_colorbar)[:,:,:3])
ax[1,1].imshow(disp_grid_0_mid, interpolation="nearest")
ax[1,2].imshow(disp_grid_1_mid, interpolation="nearest")
#ax[0,3].imshow(my_cmap(my_colorbar)[:,:,:3])

ax[2,0].imshow(1.0 - my_cmap(my_colorbar)[:,:,:3])
ax[2,1].imshow(disp_grid_0_end, interpolation="nearest")
ax[2,2].imshow(disp_grid_1_end, interpolation="nearest")
#ax[0,3].imshow(my_cmap(my_colorbar)[:,:,:3])



for xxyy, abc in zip(range(12), " ab cd ef"):
    
    xx = xxyy // 3
    yy = xxyy % 3
    
    if yy == 0:
        ax[xx,yy].set_yticks([1023, 511, 0])
        ax[xx,yy].set_xticks([0.0, 1.0])
        ax[xx,yy].set_yticklabels([0, 0.5, 1.0], fontsize=6)
        ax[xx,yy].set_xticklabels("")
    elif yy == 3:
        ax_twin = ax[xx,yy].twinx()
        ax_twin.set_yticks([0.0, 0.5, 1.0])
        ax_twin.set_xticks([0.0, 1.0])
        ax_twin.set_yticklabels([0, 4.0, 8.0], fontsize=6)
        
        ax_twin.set_xticklabels("")
        ax[xx,yy].set_xticklabels("")
        ax[xx,yy].set_yticklabels("")
    else:
        ax[xx,yy].set_xticklabels("")
        ax[xx,yy].set_yticklabels("")
        ax[xx,yy].set_xticks([disp_grid_0_start.shape[2]//2])
    
        ax[xx,yy].annotate(abc, [19,54], fontsize=14,  color=[0.9, 0.9, 0.9])
        ax[xx,yy].annotate(abc, [21,56], fontsize=14,  color=[0.9, 0.9, 0.9])
        ax[xx,yy].annotate(abc, [19,56], fontsize=14,  color=[0.9, 0.9, 0.9])
        ax[xx,yy].annotate(abc, [21,54], fontsize=14,  color=[0.9, 0.9, 0.9])
        ax[xx,yy].annotate(abc, [20,55], fontsize=14, color=[0.5, 0.125, 0.125])
    if abc == "c" or abc == "e":
        
        ax[xx,yy].annotate(abc, [19,54], fontsize=15,  color=[0.99, 0.99, 0.99])
        ax[xx,yy].annotate(abc, [21,56], fontsize=15,  color=[0.99, 0.99, 0.99])
        ax[xx,yy].annotate(abc, [19,56], fontsize=15,  color=[0.99, 0.99, 0.99])
        ax[xx,yy].annotate(abc, [21,54], fontsize=15, color=[0.99, 0.99, 0.99])
        ax[xx,yy].annotate(abc, [20,55], fontsize=14, color=[0.99, 0.11, 0.11])

ax[0,1].set_title("Hydrogeminium \n (Lenia)", fontsize=8)
ax[0,2].set_title("s613 \n (Glaberish)", fontsize=8)

#plt.tight_layout()
#plt.savefig("../assets/glaberish/gem_s613.png", bbox_inches = "tight")
plt.show()

# Figure 8: Mean-normalized average cell values over time

In [ ]:
fig, ax = plt.subplots(2,1, figsize=(3,2.5))

# mean cell values are normalized to the mean value over time. 

for iteration in range(len(mean_gems)): #num_iter):
    
    mean_gem = mean_gems[iteration]
    mean_s613 = mean_s613s[iteration]
    
    color_gem = 1.0 - np.array(my_cmap(256- iteration*10))[:3]
    color_s613 = 1.0 - np.array(my_cmap(128+iteration*10))[:3]

    ax[0].plot(mean_gem, label="Hydrogeminium", lw=.5, color=color_gem, alpha=0.25)

    ax[1].plot(mean_s613, label="s613", lw=.5, color=color_s613, alpha=0.25)
   
    if iteration == 0:
        ax[1].legend(fontsize=8, loc=4)

    if iteration == 0:
        ax[0].legend(fontsize=8, loc=4)
        
    print(f" std. dev. s613 = {np.std(mean_s613[256:])}, gem.  {np.std(mean_gem[256:])}")
    

ax[0].axis([-10, 522, 0, 1.5])
ax[1].axis([-10, 522, 0, 1.5])
ax[0].annotate("a", (522, .75), color="k", fontsize=14)
ax[1].annotate("b", (522, 0.75), color="k", fontsize=14)

#ax[0].set_xticks([0, 256, 512])
ax[0].set_xticklabels("")#[0, steps//2, 512], fontsize=6)
ax[1].set_xticks([0, steps//2, steps])
ax[1].set_xticklabels([0, steps//2, steps*2], fontsize=6)

ax[0].set_yticks([0, 0.75, 1.5])
ax[0].set_yticklabels([0, 0.75, 1.5], fontsize=6)
ax[1].set_yticks([0, 0.75, 1.5])
ax[1].set_yticklabels([0, 0.75, 1.5], fontsize=6)

 
ax[1].set_xlabel("step", fontsize=8)


plt.suptitle("Mean-normalized average cell values", fontsize=8)

#plt.tight_layout()
#ax[0].set_ylabel("mean cell value (normalized)", fontsize=8)
#plt.savefig("../assets/glaberish/average_values.png", bbox_inches="tight")

plt.show()    

# Figure 9: Spatial entropy maps of step 512 of CA simulation, starting from a bounded random uniform initial grid.

In [ ]:
# Compute spatial entropy
reload(yuca.analyze)

ca.restore_config("s613.npy")
print(f"window size for spatial entropy: {ca.neighborhood_kernels.shape}")
print("compute spatial entropy for grids 1")
spatial_h1_images = [yuca.analyze.get_spatial_entropy(elem.cpu().squeeze().numpy(), window_size=ca.neighborhood_kernels.shape[-1]) \
        for elem in [grid_1_start, grid_1_mid, grid_1_end]]


ca.restore_config("geminium2.npy")
print(f"window size for spatial entropy: {ca.neighborhood_kernels.shape}")
print("compute spatial entropy for grids 0")
spatial_h0_images = [yuca.analyze.get_spatial_entropy(elem.cpu().squeeze().numpy(), window_size=ca.neighborhood_kernels.shape[-1]) \
        for elem in [grid_0_start, grid_0_mid, grid_0_end]]


In [ ]:

fig, ax = plt.subplots(3,4, figsize=(3.25,4.5), gridspec_kw={'width_ratios': [0.25, 2, 2, 0.25]})

disp_grid_00 = 1.0 - my_cmap(input_grid.cpu().squeeze())[:,:,:3]
disp_grid_01 = spatial_h0_images[0]#.cpu().numpy().squeeze()

disp_grid_10 = 1.0 - my_cmap(grid_0_end.cpu().squeeze())[:,:,:3]
disp_grid_11 = spatial_h0_images[2]

disp_grid_20 = 1.0 - my_cmap(grid_1_end.cpu().squeeze())[:,:,:3]
disp_grid_21 = spatial_h1_images[2]

my_colorbar = np.arange(1024,0,-1).reshape(1024,1)/1024. * np.ones((1024,96))


ax[0,0].imshow(1.0 - my_cmap(my_colorbar)[:,:,:3])
ax[0,1].imshow(disp_grid_00)
ax[0,2].imshow(disp_grid_01, cmap="magma", vmin=0, vmax=8)
ax[0,3].imshow(my_cmap(my_colorbar)[:,:,:3])

ax[1,0].imshow(1.0 - my_cmap(my_colorbar)[:,:,:3])
ax[1,1].imshow(disp_grid_10)
ax[1,2].imshow(disp_grid_11, cmap="magma", vmin=0, vmax=8)
ax[1,3].imshow(my_cmap(my_colorbar)[:,:,:3])

ax[2,0].imshow(1.0 - my_cmap(my_colorbar)[:,:,:3])
ax[2,1].imshow(disp_grid_20)
ax[2,2].imshow(disp_grid_21, cmap="magma", vmin=0, vmax=8)
ax[2,3].imshow(my_colorbar, cmap=my_cmap)



for xxyy, abc in zip(range(12), " ab  cd  ef "):
    
    xx = xxyy // 4
    yy = xxyy % 4
    
    if yy == 0:
        ax[xx,yy].set_yticks([1023, 511, 0])
        ax[xx,yy].set_xticks([0.0, 1.0])
        ax[xx,yy].set_yticklabels([0, 0.5, 1.0], fontsize=6)
        ax[xx,yy].set_xticklabels("")
    elif yy == 3:
        ax_twin = ax[xx,yy].twinx()
        ax_twin.set_yticks([0.0, 0.5, 1.0])
        ax_twin.set_xticks([0.0, 1.0])
        ax_twin.set_yticklabels([0, 4.0, 8.0], fontsize=6)
        
        ax_twin.set_xticklabels("")
        ax[xx,yy].set_xticklabels("")
        ax[xx,yy].set_yticklabels("")
    else:
        ax[xx,yy].set_xticklabels("")
        ax[xx,yy].set_yticklabels("")
        ax[xx,yy].set_xticks([disp_grid_00.shape[2]//2])
    
        ax[xx,yy].annotate(abc, [19,54], fontsize=14,  color=[0.9, 0.9, 0.9])
        ax[xx,yy].annotate(abc, [21,56], fontsize=14,  color=[0.9, 0.9, 0.9])
        ax[xx,yy].annotate(abc, [19,56], fontsize=14,  color=[0.9, 0.9, 0.9])
        ax[xx,yy].annotate(abc, [21,54], fontsize=14,  color=[0.9, 0.9, 0.9])
        ax[xx,yy].annotate(abc, [20,55], fontsize=14, color=[0.5, 0.125, 0.125])
    
    if abc == "c":
        
        ax[xx,yy].annotate(abc, [19,54], fontsize=15,  color=[0.99, 0.99, 0.99])
        ax[xx,yy].annotate(abc, [21,56], fontsize=15,  color=[0.99, 0.99, 0.99])
        ax[xx,yy].annotate(abc, [19,56], fontsize=15,  color=[0.99, 0.99, 0.99])
        ax[xx,yy].annotate(abc, [21,54], fontsize=15, color=[0.99, 0.99, 0.99])
        ax[xx,yy].annotate(abc, [20,55], fontsize=14, color=[0.99, 0.11, 0.11])

#plt.tight_layout()
#plt.savefig("../assets/glaberish/spatial_entropy.png")

# Figure 10: Spatial entropy over time.

In [ ]:
temp_h0 = [elem[-1] for elem in spatial_h0s]
temp_h1 = [elem[-1] for elem in spatial_h1s]

entropy_message_gem = f"\nover {num_iter}  runs of {steps*2} steps, \n"\
        f"    Hydrogeminium had an average of {np.mean(temp_h0)} +/- {np.std(temp_h0)} (s.d) bits of entropy"


entropy_message_s613 = f"\nover {num_iter} runs of {steps*2} steps, \n"\
        f"    CA s613 had an average final v of {np.mean(temp_h1)} +/- {np.std(temp_h1)} (s.d) bits of entropy"

print(entropy_message_gem)
print(entropy_message_s613)

In [ ]:
# plots of spatial entropy
my_step = []
s_h0 = []
s_h1 = []

for hh in range(len(spatial_h0s)):
    spatial_h0 = spatial_h0s[hh]
    spatial_h1 = spatial_h1s[hh]

    for ii in range(len(spatial_h0)-1):
        my_step.append(ii * calculate_every)
        s_h0.append([np.mean(spatial_h0[ii]), np.std(spatial_h0[ii])])
        s_h1.append([np.mean(spatial_h1[ii]), np.std(spatial_h1[ii])])

    my_step.append(steps)
    s_h0.append([np.mean(spatial_h0[-1]), np.std(spatial_h0[-1])])
    s_h1.append([np.mean(spatial_h1[-1]), np.std(spatial_h1[-1])])

    color_0 = 1.0 - np.array(my_cmap(96))[0:3]
    color_1 = 1.0 - np.array(my_cmap(192))[0:3]

    s_h0_mean = [elem[0] for elem in s_h0]
    s_h1_mean = [elem[0] for elem in s_h1]


    s_h0_low = [elem[0] - elem[1] for elem in s_h0]
    s_h0_high = [elem[0] + elem[1] for elem in s_h0]
    s_h1_low = [elem[0] - elem[1] for elem in s_h1]
    s_h1_high = [elem[0] + elem[1] for elem in s_h1]

    fig, ax = plt.subplots(1,1, figsize=(3,2))
    ax.set_title("Spatial Entropy $\pm$ standard deviation", fontsize=8)

    ax.plot(my_step, s_h0_mean, label="Hydrogeminium", color=color_0)
    ax.plot(my_step, s_h1_mean, label="s613", color=color_1)

    ax.fill_between(my_step, s_h0_low, s_h0_high, alpha=0.35, color=color_0)
    ax.fill_between(my_step, s_h1_low, s_h1_high, alpha=0.35, color=color_1)

    ax.set_xlabel("step", fontsize=6)
    ax.set_ylabel("Spatial entropy (bits)", fontsize=6)
    ax.legend(fontsize=7)

    ax.set_xticks([0,steps//2, steps])
    ax.set_xticklabels([0, steps//2, steps], fontsize=7)

    ax.set_yticks(np.arange(-1,9))
    ax.set_yticklabels(np.arange(-1,9), fontsize=7)


    #plt.savefig(f"../assets/glaberish/spatial_entropy_plot{hh}.png")
    plt.show()

# Figure 11: Gliders in Hydrogeminium and s613

In [ ]:
# Figure showing examples of mobile patterns in s613 and Hydrogeminium
lib = Librarian()
lib.update_index()


fig, ax = plt.subplots(2, 2, figsize=(3.8, 3))

torch.set_default_dtype(torch.float32)
dim_x = 120
dim_y = 200

grid_s0 = np.zeros((1, 1, dim_x, dim_y))
grid_s1 = np.zeros((1, 1, dim_x, dim_y))

grid_g0 = np.zeros((1, 1, dim_x, dim_y))
grid_g1 = np.zeros((1, 1, dim_x, dim_y))


geminium, gem_metadata = lib.load("geminium_hydrogeminium_natans000")
gem_config = gem_metadata["ca_config"]
gemwobbler, _ = lib.load("geminium2_wobble_glider000")

frog, s613_metadata = lib.load("s613_s613_frog000")
s613_config = s613_metadata["ca_config"]
wobbler, _ = lib.load("s613_fast_wobble_glider000")

ca_gem = CA()
ca_s613 = CA()

ca_gem.restore_config("geminium.npy")
ca_s613.restore_config("s613.npy")
ca_gem.no_grad()       
ca_s613.no_grad()

ca_gem.to_device(my_device)
ca_s613.to_device(my_device)

glider_steps = 64

grid_g0[0,0, 60:60+geminium.shape[-2], :geminium.shape[-1]] = geminium
grid_g1[0,0, 40:40+gemwobbler.shape[-2], 59:59+gemwobbler.shape[-1]] = gemwobbler
                       

grid_s0[0,0,:wobbler.shape[-2], :wobbler.shape[-1]] = wobbler
grid_s1[0,0, 80:80+frog.shape[-2], 0+wobbler.shape[-1]:0+wobbler.shape[-1]+frog.shape[-1]] = frog
                       

## update for steps

grid_g0 = torch.tensor(grid_g0).to(torch.get_default_dtype()).to(my_device)
grid_g1 = torch.tensor(grid_g1).to(torch.get_default_dtype()).to(my_device)

grid_s0 = torch.tensor(grid_s0).to(torch.get_default_dtype()).to(my_device)
grid_s1 = torch.tensor(grid_s1).to(torch.get_default_dtype()).to(my_device)

com_g0 = []
com_g1 = []
com_s0 = []
com_s1 = []
    
for step in range(glider_steps):
    
    grid_s0 = ca_s613(grid_s0)
    grid_s1 = ca_s613(grid_s1)
    grid_g0 = ca_gem(grid_g0)
    
    com_g0.append(get_center_of_mass(grid_g0))
    grid_g0 = ca_gem(grid_g0)
    
    grid_g1 = ca_gem(grid_g1)
    
    com_g0.append(get_center_of_mass(grid_g0))
    com_g1.append(get_center_of_mass(grid_g1))
    com_s0.append(get_center_of_mass(grid_s0))
    com_s1.append(get_center_of_mass(grid_s1))

    

ax[0,0].set_title("Hydrogeminium \n patterns", fontsize=8)

ax[0,1].set_title("s613 \n patterns", fontsize=8)

disp_s0 = 1.0 - my_cmap(grid_s0.squeeze())[:,:,:3]
disp_s1 = 1.0 - my_cmap(grid_s1.squeeze())[:,:,:3]
disp_g0 = 1.0 - my_cmap(grid_g0.squeeze())[:,:,:3]
disp_g1 = 1.0 - my_cmap(grid_g1.squeeze())[:,:,:3]

ax[0,0].imshow(disp_g0)
ax[0,1].imshow(disp_s0)
ax[1,0].imshow(disp_g1)
ax[1,1].imshow(disp_s1)


my_green = 1.0 - np.array(my_cmap(grid_g0.cpu().max().numpy()))[:3]
my_blue = 1.0 - np.array(plt.get_cmap("afmhot")(grid_g0.cpu().max().numpy()))[:3]


for ii in range(len(com_g0)):
    progress = ii / len(com_g0)
    my_color = progress * my_green + (1.0 - progress) * my_blue
    ax[0,0].scatter(com_g0[ii][0],com_g0[ii][1], color=my_color, s=1.0, alpha=0.05) 

for ii in range(len(com_g1)):
    progress = ii / len(com_g1)
    my_color = progress * my_green + (1.0 - progress) * my_blue
    ax[1,0].scatter(com_g1[ii][0],com_g1[ii][1], color=my_color, s=1.0, alpha=0.05) 
    
    
for ii in range(len(com_s0)):
    progress = ii / len(com_s0)
    my_color = progress * my_green + (1.0 - progress) * my_blue
    ax[0,1].scatter(com_s0[ii][0],com_s0[ii][1], color=my_color, s=1.0, alpha=0.05) 

for ii in range(len(com_s1)):
    progress = ii / len(com_s1)
    my_color = progress * my_green + (1.0 - progress) * my_blue
    ax[1,1].scatter(com_s1[ii][0],com_s1[ii][1], color=my_color, s=1.0, alpha=0.05) 
        
for xxyy, abc in zip(range(4), "abcd"):
    xx = xxyy // 2
    yy = xxyy % 2
    
    ax[xx, yy].set_xticklabels("")
    ax[xx, yy].set_yticklabels("")
    ax[xx,yy].annotate(abc, (10,20), fontsize=12)


plt.tight_layout()
#plt.savefig("../assets/glaberish/gem_s613_patterns.png", bbox_inches = "tight")
plt.show()

# Extra experimental figures: conditional entropy measures

In [ ]:
fig1, ax1 = plt.subplots(1,1, figsize=(4.5,2))

fig2, ax2 = plt.subplots(1,1, figsize=(4.5,2))

fig3, ax3 = plt.subplots(1,1, figsize=(4.5,3))

for gg in range(1):

    color_gem = 1.0 - np.array(my_cmap(64))[:3]
    color_s613 = 1.0 - np.array(my_cmap(128))[:3]

    color_gem2 = 1.0 - np.array(my_cmap(96))[:3]
    color_s6132 = 1.0 - np.array(my_cmap(192))[:3]

    ax1.plot(h_cond_step, h_gems[gg], label="Hydrogeminium", color=color_gem2, alpha=0.5)
    ax1.plot(h_cond_step, h_s613s[gg], label="s613", color=color_s6132, alpha=0.5)

    #plt.savefig("../assets/glaberish/entropy_values.png")

    ax2.plot(h_cond_step, h_cond_gems_2[gg], label="Hydrogeminium", color=color_gem, alpha=0.5)
    ax2.plot(h_cond_step, h_cond_s613s_2[gg], label="s613", color=color_s613, alpha=0.5)

    plt.tight_layout()
    #plt.savefig("assets/average_values.png")

    ax3.plot(h_cond_gems[gg], label="Conditional entropy Hydrogeminium", color=color_gem, alpha=0.5)
    ax3.plot(h_cond_s613s[gg], label = "Conditional entropy s613", color=color_s613, alpha=0.5)
    ax3.plot(h_baselines * len(h_cond_s613), label = "Conditional entropy, random baseline")

    print(f"statistics for {gg}th metrics (after initial 256 steps):")
    print(f" std. dev. s613 = {np.std(mean_s613s[gg][256:])}, gem.  {np.std(mean_gems[gg][256:])}")

    if gg == 0:

        ax1.set_xlabel("step", fontsize=4)
        ax1.set_ylabel("entropy (bits)", fontsize=4)
        ax1.set_title("Image entropy", fontsize=6)
        ax1.legend(fontsize=6)

        ax1.set_yticks([np.min(h_s613), np.max(h_s613)])
        ax1.set_yticklabels([f"{np.min(h_s613):.4f}", f"{np.max(h_s613):.4f}"], fontsize=6)
        ax1.set_xticks(h_cond_step[0::2])
        ax1.set_xticklabels(h_cond_step[0::2], fontsize=6)

        ax2.legend(fontsize=6,loc=4)
        ax2.set_ylabel("entropy, bits", fontsize=6)
        ax2.set_xlabel("step", fontsize=6)

        ax2.set_xticks(h_cond_step[0::2])
        ax2.set_xticklabels(h_cond_step[0::2], fontsize=6)

        ax2.set_yticks([np.min(h_cond_s613_2), np.max(h_cond_s613_2)])
        ax2.set_yticklabels([f"{np.min(h_cond_s613_2):.4f}", f"{np.max(h_cond_s613_2):.4f}"], fontsize=6)

        ax2.set_title("Conditional Entropy (relative to random input -> $\Gamma$)", fontsize=6)
        ax2.legend(fontsize=6)

        ax3.set_title("Conditional entropy", fontsize=7)
        ax3.set_ylabel("entropy, bits", fontsize=6)
        ax3.set_xlabel("step", fontsize=6)
        ax3.legend(fontsize=6)
        ax3.set_xticks(np.arange(0,len(h_cond_gem),4))
        ax3.set_xticklabels(np.arange(0, len(h_cond_gem),4), fontsize=6)
        ax3.set_yticks([7.9925, 7.995, 7.9975, 8.0])
        ax3.set_yticklabels([7.9925, 7.995, 7.9975, 8.0],fontsize=6)
        
    #plt.savefig("assets/average_values.png")


plt.show()


In [ ]:
# Plotting conditional entropy based measure ($\Gamma$) from Pena and Sayama, 2021.

plt.figure(figsize=(3,1))
plt.plot(h_cond_gem_2, label="$\Gamma$ Hydrogeminium")
plt.plot(h_cond_s613_2, label = "$\Gamma$ s613")
plt.legend(fontsize=6)
plt.xticks(fontsize=6)
plt.yticks(fontsize=6)

plt.ylabel("$\Gamma$ (bits)", fontsize=6)
plt.xlabel("step", fontsize=6)
plt.show()

plt.figure(figsize=(4.5,1.5))
plt.plot(h_cond_gem, label="Conditional entropy Hydrogeminium")
plt.plot(h_cond_s613, label = "Conditional entropy s613")
plt.plot(h_baselines * len(h_cond_s613), label = "Conditional entropy, random baseline")

plt.legend(fontsize=6,loc=4)
plt.xticks(fontsize=6)
plt.yticks(fontsize=6)
plt.ylabel("entropy, bits", fontsize=6)
plt.xlabel("step", fontsize=6)
plt.show()

In [ ]:
fig, ax = plt.subplots(1,1, figsize=(3,2))


color_gem = 1.0 - np.array(my_cmap(64))[:3]
color_s613 = 1.0 - np.array(my_cmap(128))[:3]

color_gem2 = 1.0 - np.array(my_cmap(96))[:3]
color_s6132 = 1.0 - np.array(my_cmap(192))[:3]

#ax.plot(h_gem, label="Hydrogeminium", color=color_gem, alpha=0.5)
#ax.plot(h_s613, label="s613", color=color_s613, alpha=0.5)


ax.plot(h_cond_step, h_gem, label="Hydrogeminium", color=color_gem2, alpha=0.5)
ax.plot(h_cond_step, h_s613, label="s613", color=color_s6132, alpha=0.5)

plt.xlabel("step", fontsize=4)
plt.ylabel("", fontsize=4)

#ax.set_xticklabels("")
#ax.set_yticklabels("")

plt.title("Entropy", fontsize=6)
plt.legend(fontsize=6)
#plt.savefig("assets/average_values.png")

plt.tight_layout()
fig, ax = plt.subplots(1,1, figsize=(3,2))


ax.plot(h_cond_step, h_cond_gem_2, label="Hydrogeminium", color=color_gem, alpha=0.5)
ax.plot(h_cond_step, h_cond_s613_2, label="s613", color=color_s613, alpha=0.5)

plt.xlabel("step", fontsize=4)
plt.ylabel("mean cell value (normalized)", fontsize=4)

#ax.set_xticklabels("")
#ax.set_yticklabels("")

plt.title("Conditional Entropy (relative to random input)", fontsize=6)
plt.legend(fontsize=6)
#plt.savefig("assets/average_values.png")

plt.tight_layout()
#plt.savefig("assets/average_values.png")

plt.show()

print(f" std. dev. s613 = {np.std(mean_s613[256:])}, gem.  {np.std(mean_gem[256:])}")

#### math symbols
$$
dt
$$

$$
A^{t+dt} = A^{t} + dt \cdot G(K \ast A^{t})
$$

$$
A^{t + dt}
$$

$$
t + dt
$$

$$
t
$$

$$
G(\cdot)
$$

$$
P(\cdot)
$$


$$
G_{gen}(\cdot)
$$

$$
K
$$

$$
A^{t}
$$

$$
A^{t+dt} = s(K \ast A^t, A^t)
$$

$$
A^{t+dt} = A^{t} + dt \left[ (1-A^t)  G_{gen}(K \ast A^t) + A^t P(K \ast A^t) \right]
$$

$$
s(\cdot)
$$

$$
K \ast A^t
$$

$$
0.1
$$


$$
A^{t+dt} = A^t + f(A^t)dt
$$

$$
\frac{dA^{t}}{dt} = f(A^t)
$$

In [ ]:
# make a colorbar 

import matplotlib.pyplot as plt

fig, ax = plt.subplots(1,1, figsize =(2., .5 ), facecolor="white")

my_colorbar = (np.arange(0,1024) / 1024.) * np.ones((128,1024))

print(my_colorbar.max())

my_colorbar = 1.0 - my_cmap(my_colorbar)[:,:,:3]

ax.imshow(my_colorbar)

ax.set_xticks([0, 511, 1023])
ax.set_xticklabels([0., 0.5, 1.0], {"fontsize": 5})
ax.set_yticklabels('')

plt.tight_layout()
#plt.savefig("../assets/glaberish/h_colorbar3.png")
plt.show()
